# Construção da base de notícias ESG a partir da captura do Google RSS

- Autor: Daniel Saraiva Leite - 2023
- Projeto Análise de sentimentos sobre notícias do tema ESG
- Trabalho de conclusão de curso - MBA Digital Business USP Esalq

In [1]:
import warnings
import nltk
import numpy as np
import pandas as pd
import re
import unidecode
import datetime as dt
from noticias_google_buscador import busca_noticias_google_news
from noticias_processamento_texto import remove_acentos, remove_termos_comuns, aplica_stemming_texto, remove_palavras_texto, conta_termos_esg, classifica_texto, classifica_textos_coletados, filtra_noticias_nao_relacionadas, filtra_noticias_sem_classificacao, conta_mencoes_empresas, filtra_citacao_relevante, remove_nome_composto
from noticias_google_buscador_esg import busca_noticias_google_periodo, busca_noticias_google_esg, filtra_noticias_sites_especificos, recupera_noticias_completas, processa_busca_empresa, recupera_lista_empresas_B3


warnings.filterwarnings('ignore')
arquivo_termos_esg = 'datasets/palavras_chave_esg.xlsx'
base_noticias = 'datasets/base_noticias.xlsx'



'''
 Gera um arquivo em excel com todas as empresas da B3 processadas
'''
def processa_base_noticias():
    
    dfEmpresasListadas = recupera_lista_empresas_B3()
    
    # seleciona apenas do ranking ISE
    dfEmpresasListadas = dfEmpresasListadas[~ pd.isnull(dfEmpresasListadas.RANKING) ]

    list_empresas = list(set(list(dfEmpresasListadas['Nome'])))

    
    list_empresas = [remove_acentos(i.lower()) for i in list_empresas]
    
    df_atual = pd.read_excel(base_noticias)
    
    for empresa in list_empresas:
        
        if len(df_atual[df_atual['empresa'] == empresa]) == 0:  # inclusao
        
            print ('empresa: ' + empresa + ' - ' + str(dt.datetime.now()))
            df = processa_busca_empresa(empresa, dfEmpresasListadas)
            
            if len(df) > 0:
                print('Adicionadas ' + str(len(df)) + ' notícias para ' + empresa  )
                df = pd.concat([df_atual,df]).drop_duplicates()
                df_atual = df

        df_atual = df_atual.drop_duplicates(['titulo','empresa','fonte', 'texto_completo'], keep='last')
        df_atual.to_excel(base_noticias, index=False)
            
    return df_atual



[nltk_data] Downloading package rslp to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# processa todas as empresas da lista da B3 com ISE
#processa_base_noticias()

In [3]:
# processa uma empresa especificada
for empresa in [ 'c&a']:
    dfEmpresasListadas = recupera_lista_empresas_B3()
    df1 = busca_noticias_google_esg(empresa)
    df2 = recupera_noticias_completas(df1)
    df3 = filtra_noticias_nao_relacionadas(df2, empresa)
    df4 = filtra_citacao_relevante(df3, empresa, dfEmpresasListadas )
    df5 = classifica_textos_coletados(df4)
    df6 = df5
    df_atual = pd.read_excel(base_noticias)
    
    # corte de data - se necessario inibir noticias alem de uma data
    df6 = df6[df6.data_publicacao.dt.date < dt.date(2023, 3, 1)]
    
    df = pd.concat([df_atual,df6]).drop_duplicates(ignore_index=True)
    df = df.drop_duplicates(['titulo','empresa','fonte', 'texto_completo'], keep='last')
    df.to_excel(base_noticias, index=False)
    



    